In [1]:
import numpy as np
import os
import sys
import cv2
import urllib2
import urllib

In [2]:
home_dir = os.getenv("HOME")
caffe_root = os.path.join(home_dir, 'caffe')
sys.path.insert(0, os.path.join(caffe_root, 'python'))

import caffe

In [3]:
class NetFace:
    
    def __init__(self):
        path = "haarcascade_frontalface_alt.xml"
        self.classifier = cv2.CascadeClassifier(path)
    def classifierImage(self,miniframe):
        return self.classifier.detectMultiScale(miniframe)

In [4]:
class Face:
    
    def __init__(self,net=NetFace()):
        self.net = net

    def detect(self,frame):
        height, width, depth = frame.shape

        # create grayscale version
        grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # equalize histogram
        cv2.equalizeHist(grayscale, grayscale)

        # detect objects
        DOWNSCALE = 4
        minisize = (frame.shape[1]/DOWNSCALE,frame.shape[0]/DOWNSCALE)
        miniframe = cv2.resize(frame, minisize)
        faces = self.net.classifierImage(miniframe)
        
        if len(faces)>0:
            print 'Face detected!'

            for i in faces:
                x, y, w, h = [ v*DOWNSCALE for v in i ]
                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
                cv2.imshow('frame',frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
                #print("Faces: ", len(faces))
        
        return faces

In [5]:
class Net:

    def __init__(self):
        self.create_net()
        self.create_mu()
        self.create_transformer()
        self.create_labels()
        self.loadsynset()

    def create_transformer(self):
        self.transformer = caffe.io.Transformer({'data': self.net.blobs['data'].data.shape})
        self.transformer.set_transpose('data', (2,0,1))
        self.transformer.set_mean('data', self.mu)      
        self.transformer.set_raw_scale('data', 255)      
        self.transformer.set_channel_swap('data', (2,1,0)) 

    def create_mu(self):
        self.mu = np.load(os.path.join(caffe_root, 'python','caffe','imagenet','ilsvrc_2012_mean.npy'))
        self.mu = self.mu.mean(1).mean(1)  

    def create_net(self):
        model_weights = os.path.join(caffe_root, 'models','bvlc_reference_caffenet','bvlc_reference_caffenet.caffemodel')
        model_def = os.path.join(caffe_root, 'models', 'bvlc_reference_caffenet','deploy.prototxt')
        self.net = caffe.Net(model_def,model_weights,caffe.TEST)

    def create_labels(self):
        labels_file = os.path.join(caffe_root, 'data','ilsvrc12','synset_words.txt')
        self.labels = np.loadtxt(labels_file, str, delimiter='\t') 

    def loadsynset(self):
        f = open("synset_cats","r")
        self.cats = f.read().splitlines()
        f.close()
        
        f = open("synset_dogs","r")
        self.dogs = f.read().splitlines()
        f.close()

    def predict_imageNet(self,image_filename):
        image = caffe.io.load_image(image_filename)
        self.net.blobs['data'].data[...] = self.transformer.preprocess('data', image)

        # perform classification
        self.net.forward()

        # obtain the output probabilities
        output_prob = self.net.blobs['prob'].data[0]

        # sort top five predictions from softmax output
        top_inds = output_prob.argsort()[::-1][:5]

        predictions = zip(output_prob[top_inds], self.labels[top_inds])

        return predictions

    def result(self,img):
        predictions = self.predict_imageNet(img)
        total_dogs = 0
        total_cats = 0
        
        for per, cls in predictions:
            if cls.split()[0] in self.cats:
                total_cats += per
            
            elif cls.split()[0] in self.dogs:
                total_dogs += per
            
            return total_dogs*100,total_cats*100

In [6]:
class Input:

    FACES = 1
    PREDICTION = 2
    
    def __init__(self, load=None):
        self.img = None
        if(load): self.resolve(load)

    def resolve(self,load):
        if(self.isFile(load)):
            self.fileResolver(load)	
        
        elif(self.isUrl(load)):
            self.urlResolver(load)

    def isFile(self,load):
        return os.path.isfile(load)
    
    def isUrl(self,load):
        try:
            urllib2.urlopen(load)
            return True
        
        except urllib2.HTTPError, e:
            return False

        except urllib2.URLError, e:
            return False

        return False

    def fileResolver(self,load):
        self.img = cv2.imread(load)
        self.load = load

    def urlResolver(self,load):
        image = urllib.URLopener()
        path = os.path.join("Images","001.jpg")
        image.retrieve(load,path)
        image.close()
        self.fileResolver(path)

        return None

    def getImage(self,destination):
        if(destination == self.FACES):
            return self.img

        else:
            return self.load

In [7]:
class Output:

    def outFaces(self,faces):
        if(len(faces) > 0):
            print("Foram detectadas {0} faces").format(str(len(faces)))
            print("As coordendas das faces: ")

            for face in faces:
                print(face)
        
        else:
            print("Nao foram encontradas faces")

        
    def outAnimals(self,dogs,cats):
        if(dogs > 0):
            print("A probabilidade de haver caninos na imagem e de: {0}").format(str(dogs))

        else:
            print("Nao ha caninos na imagem")

        if(cats > 0):
            print("A probabilidade de haver felinos na imagem e de: {0}").format(str(cats))

        else:
            print("Nao ha felinos na imagem")

In [8]:
def main(argv=sys.argv):
    f = Face()
    n = Net()
    o = Output()
    
    if(len(argv) >= 2):
        argv.pop(0)

        for arg in argv:
            print(arg)
            d = Input(arg)
            o.outFaces(f.detect(d.getImage(Input.FACES)))
            dogs,cats = n.result(d.getImage(Input.PREDICTION))
            o.outAnimals(dogs, cats)


In [9]:
img_dir = 'Images'
images = os.listdir(img_dir)

In [10]:
for img_file in images:
    main([" ", os.path.join(img_dir, img_file)])
    print "\n"

Images/004.jpg
Nao foram encontradas faces
Nao ha caninos na imagem
A probabilidade de haver felinos na imagem e de: 31.2447071075


Images/002.jpg
Face detected!
Foram detectadas 3 faces
As coordendas das faces: 
[129  22  31  31]
[85 94 45 45]
[138  97  64  64]
Nao ha caninos na imagem
Nao ha felinos na imagem


Images/003.jpg
Nao foram encontradas faces
Nao ha caninos na imagem
Nao ha felinos na imagem


Images/001.jpg
Face detected!
Foram detectadas 1 faces
As coordendas das faces: 
[79 28 33 33]
Nao ha caninos na imagem
Nao ha felinos na imagem


